In [ ]:
#Setup packages and sqlite connection
import sqlite3 as sq
import pandas as pd
import numpy as np
cnx = sq.connect('test.db')
cur = cnx.cursor()

In [ ]:
# Check tables active in the DB
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()       

## One-Time DB Creation Steps

In [ ]:
#Begin here only if need to update the DB files
#Read CSV File in to temp dataframe and check shape
tempdf = pd.read_csv("pacs2studylevelcheck.csv")
tempdf.shape

In [ ]:
#Check it looks as exepected
tempdf.head()

In [ ]:
#Write to database for easier startup in the future
#Replace databasename and write rules (currently replaces what is there) as needed
tempdf.to_sql('pacs1premigrationcheck', cnx, if_exists='replace', index=False)

## Dataframe Analysis

In [ ]:
# Check tables active in the DB
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()       

In [ ]:
#Start here if sql database already setup
#Read database in to pandas dataframe and check shape
df = pd.read_sql_query("""SELECT * FROM pacs1premigrationcheck;""", cnx)
df.shape

In [ ]:
#Check it looks as expected
df.head()

In [ ]:
df.dtypes

In [ ]:
#Clean-up Data
df.replace(np.NaN,'NONE',inplace=True)

In [ ]:
#Sort Data to get information with most images in pacs2
df.sort_values(['NUM_OBJECTS_STUDY_PACS2'], ascending=False, inplace=True)

In [ ]:
#Use for Deidentified Data
#Remove duplicate suid to get to Study-Level Analysis
dfdeidentified = df.drop(['PATIENT_ID_PACS2','patID','ST_PATIENT_ID','ACCESSION_NUMBER_PACS2','ST_ACCESSIONNUMBER','STUDY_DATE_PACS2', 'studydate', 'ST_DATE', 'STUDY_DESCRIPTION_PACS2', 'PATIENT_NAME_PACS2','PT_DICOMFAMILYNAMECOMPLEX'], axis=1)
df1 = dfdeidentified.drop_duplicates(subset='suid',keep='first').copy()
df1.shape

In [ ]:
#Use for Non-Deidentified Data
#Remove duplicate suid to get to Study-Level Analysis
df1 = df.drop_duplicates(subset='suid',keep='first').copy()
df1.shape

In [ ]:
df1['INSTITUTIONAL_DEPARTMENT_NAME_PACS2'].value_counts()

In [ ]:
df1.dtypes

In [ ]:
df1['PATIENT_ID_PACS2'] = df1['PATIENT_ID_PACS2'].astype(str)

#df['id']= df['id'].astype(str)

In [ ]:
df1.iloc[679981,1]

In [ ]:
df1[df1['PATIENT_ID_PACS2'].apply(lambda x: len(x) !=12)]

#df['names'].apply(lambda x: len(x)>1) &

# Patient Crosswalk

In [ ]:
#Load Data from pacs1
#Set PK from pacs1 (PAT_ID) as Index
dfpatientextract = pd.read_csv('pacs1patientextract05012019.csv' 
                               ,header=0
                               ,converters={'pacs1_MRN':str, 'pacs1_MRN_9_Digits':str, 'pacs1_MRN_12_Digits':str, 'pacs1_DOB':str}
                               ,index_col = 'pacs1_PAT_ID'
                               ,parse_dates=['pacs1_DOB'], date_parser = lambda d: pd.to_datetime(d, format = '%Y%m%d', errors='coerce')
                               )
dfpatientextract.shape

In [ ]:
print(dfpatientextract.info())
dfpatientextract.head()

In [ ]:
#Load Crosswalk data from EDW team, sheet by sheet
dfcurrentmrnmatch = pd.read_excel('pacs1_Mapping.xlsx'
                                  ,sheet_name = 'Current MRN mapping'
                                  ,header=0
                                  ,index_col = 'PAT_ID'
                                  ,usecols = ['PAT_ID', 'Clarity_MRN','Clarity_PAT_NAME','Clarity_DOB','Clarity_SEX']
                                #  ,names = ['Mapped_MRN','Clarity_PAT_NAME','Clarity_DOB','Mapped_Gender']
                                  ,converters={'Clarity_MRN':str}
                                # ,parse_dates=['Clarity_DOB'], date_parser = lambda d: pd.Series.dt.strftime(d, format="%Y%m%d")
                                 )

dfcurrentmrnmatch.shape

In [ ]:
dfhistoricalmrnmatch = pd.read_excel('pacs1_Mapping.xlsx'
                                  ,sheet_name = 'Historical MRN Mapping'
                                  ,header=0
                                  ,index_col = 'PAT_ID'
                                  ,usecols = ['PAT_ID', 'Clarity_New_MRN_2','Clarity_PAT_NAME','CLARITY_DOB','Clarity_SEX']
                                 # ,names = ['PAT_ID','Mapped_MRN','Clarity_PAT_NAME','Clarity_DOB','Mapped_Gender']
                                  ,converters={'Clarity_New_MRN_2':str}
                                # ,parse_dates=['Clarity_DOB'], date_parser = lambda d: pd.Series.dt.strftime(d, format="%Y%m%d")
                                 )

dfhistoricalmrnmatch.shape

In [ ]:
print(dfcurrentmrnmatch.info())
dfcurrentmrnmatch.head()

In [ ]:
print(dfhistoricalmrnmatch.info())
dfhistoricalmrnmatch.head()

In [ ]:
dfcurrentmrnmatch.rename(columns={'Clarity_MRN':'Mapped_MRN'},inplace=True)
dfhistoricalmrnmatch.rename(columns={'Clarity_New_MRN_2':'Mapped_MRN','CLARITY_DOB':'Clarity_DOB'}, inplace=True)
dfpremapping = dfcurrentmrnmatch.append(dfhistoricalmrnmatch)
dfpremapping.info()

In [ ]:
dfpremapping[['Mapping_Last_Name','Mapping_First_Name']] = dfpremapping.Clarity_PAT_NAME.str.split(",",expand=True)

In [ ]:
dfpremapping.head()

In [ ]:
dfmapping = dfpatientextract.join(dfpremapping, how='left')
dfmapping.shape

In [ ]:
print(dfmapping.info())
dfmapping.head()

In [ ]:
dfmapping.iloc[1,3]

In [ ]:
dfhasmapping = dfmapping[~dfmapping['Mapped_MRN'].isnull()].copy()
dfhasmapping.shape

In [ ]:
dfhasmapping.head()

In [ ]:
dfhasmapping['Conflicts'] = 0
dfhasmapping['Conflict_Type'] = ''
for i in range(0,len(dfhasmapping)):
   #Last Names don't match 
    if dfhasmapping.iloc[i,3] != dfhasmapping.iloc[i,12]:
        dfhasmapping.iloc[i,14] += 1   #Increase conflict count by one
        dfhasmapping.iloc[i,15] += 'L' #Add L as a conflict type
   #First Names don't match 
    if dfhasmapping.iloc[i,4] != dfhasmapping.iloc[i,13]: 
        dfhasmapping.iloc[i,14] += 1   #Increase conflict count by one
        dfhasmapping.iloc[i,15] += 'F' #Add F as a conflict type
   #Gender doesn't match 
    if dfhasmapping.iloc[i,6] != dfhasmapping.iloc[i,11]: 
        dfhasmapping.iloc[i,14] += 1   #Increase conflict count by one
        if dfhasmapping.iloc[i,6] is None or dfhasmapping.iloc[i,6]=='MISSING' or dfhasmapping.iloc[i,6]=='':
            dfhasmapping.iloc[i,15] += 'NG' #Add NG as a conflict type  
        else:
            dfhasmapping.iloc[i,15] += 'G' #Add G as a conflict type         
   #DOB doesn't match 
    if dfhasmapping.iloc[i,7] != dfhasmapping.iloc[i,10]:
        dfhasmapping.iloc[i,14] += 1  #Increase conflict count by one
        if dfhasmapping.iloc[i,7] is None or dfhasmapping.iloc[i,7]=='MISSING' or dfhasmapping.iloc[i,7] =='':
            dfhasmapping.iloc[i,15] += 'ND' #Add D as a conflict type  
        else:
            dfhasmapping.iloc[i,15] += 'D' #Add D as a conflict type  
        

In [ ]:
print('Conflict Counts :\n', dfhasmapping.Conflicts.value_counts(dropna=False))
print('\nConflict Counts by Type : \n', dfhasmapping.Conflict_Type.value_counts(dropna=False))

In [ ]:
dfhasmapping.columns

In [ ]:
dfperfectmatch = dfhasmapping.loc[:,['pacs1_MRN','Mapped_MRN','pacs1_Last_Name','Mapping_Last_Name','pacs1_First_Name','Mapping_First_Name']][dfhasmapping['Conflicts']==0]
dfperfectmatch.to_csv("C:\\Users\\username\\Desktop\\pacs1perfectmatch.csv")

In [ ]:
dfconflicts = dfhasmapping[dfhasmapping['Conflicts']>0]
dfconflicts.shape

In [ ]:
dfconflicts.loc[:,['pacs1_MRN','Mapped_MRN','pacs1_Last_Name','Mapping_Last_Name','pacs1_First_Name','Mapping_First_Name','pacs1_Gender','Clarity_SEX','pacs1_DOB','Clarity_DOB']][dfconflicts['Conflict_Type']=='LFGD']